In [1]:
import os
from os.path import expanduser
home = expanduser("~/")

import sys
# sys.path.insert(0, '/global/u2/x/xshuang/gigalens-xh-dev/src')

sys.path.insert(0, home+'/gigalens'+'/src')
print('MASTER BRANCH GIGALENS')

srcdir = os.path.join(home, "gigalens/src/")

import jax
print(jax.devices())


MASTER BRANCH GIGALENS
[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0), StreamExecutorGpuDevice(id=1, process_index=0, slice_index=0), StreamExecutorGpuDevice(id=2, process_index=0, slice_index=0), StreamExecutorGpuDevice(id=3, process_index=0, slice_index=0)]


In [2]:
from gigalens.jax.inference import ModellingSequence
from gigalens.jax.model import ForwardProbModel, BackwardProbModel
from gigalens.model import PhysicalModel
from gigalens.jax.simulator import LensSimulator
from gigalens.simulator import SimulatorConfig
from gigalens.jax.profiles.light import sersic
from gigalens.jax.profiles.mass import epl, shear

import tensorflow_probability.substrates.jax as tfp
import jax
from jax import random
import numpy as np
import optax
from jax import numpy as jnp
from matplotlib import pyplot as plt
import optax
import corner
import yaml
import pickle
from helpers import *
tfd = tfp.distributions
jax.devices()

/global/common/software/deepsrch/jax_gigalens_kernel/jax-gigalens/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=1, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=2, process_index=0, slice_index=0),
 StreamExecutorGpuDevice(id=3, process_index=0, slice_index=0)]

In [3]:
# * Load in data
systems_dir = "SystemSaves"

f = np.load(os.path.join(systems_dir, "100Systems60px.npz"))
keys = f.files
observed_imgs = jnp.array([f[key] for key in keys])




filename = os.path.join(systems_dir, '100SystemsParams.yaml')
with open(filename, 'r') as file:
    true_params = params_lists_to_jax(yaml.safe_load(file))


In [4]:
#* Initialize Priors
lens_prior = tfd.JointDistributionSequential(
    [
        tfd.JointDistributionNamed(
            dict(
                theta_E=tfd.LogNormal(jnp.log(1.25), 0.25),
                gamma=tfd.TruncatedNormal(2, 0.25, 1, 3),
                e1=tfd.Normal(0, 0.1),
                e2=tfd.Normal(0, 0.1),
                center_x=tfd.Normal(0, 0.05),
                center_y=tfd.Normal(0, 0.05),
            )
        ),
        tfd.JointDistributionNamed(
            dict(gamma1=tfd.Normal(0, 0.05), gamma2=tfd.Normal(0, 0.05))
        ),
    ]
)
lens_light_prior = tfd.JointDistributionSequential(
    [
        tfd.JointDistributionNamed(
            dict(
                R_sersic=tfd.LogNormal(jnp.log(1.0), 0.15),
                n_sersic=tfd.Uniform(1, 6),
                e1=tfd.TruncatedNormal(0, 0.1, -0.3, 0.3),
                e2=tfd.TruncatedNormal(0, 0.1, -0.3, 0.3),
                center_x=tfd.Normal(0, 0.05),
                center_y=tfd.Normal(0, 0.05),
                Ie=tfd.LogNormal(jnp.log(100.0), 0.3),
            )
        )
    ]
)

source_light_prior = tfd.JointDistributionSequential(
    [
        tfd.JointDistributionNamed(
            dict(
                R_sersic=tfd.LogNormal(jnp.log(0.25), 0.15),
                n_sersic=tfd.Uniform(0.5, 4),
                e1=tfd.TruncatedNormal(0, 0.15, -0.5, 0.5),
                e2=tfd.TruncatedNormal(0, 0.15, -0.5, 0.5),
                center_x=tfd.Normal(0, 0.25),
                center_y=tfd.Normal(0, 0.25),
                Ie=tfd.LogNormal(jnp.log(400.0), 0.5),
            )
        )
    ]
)

prior = tfd.JointDistributionSequential(
    [lens_prior, lens_light_prior, source_light_prior]
)

In [5]:
kernel = np.load('/global/homes/l/linusu/gigalens/src/gigalens/assets/psf.npy').astype(np.float32)
sim_config = SimulatorConfig(delta_pix=0.065, num_pix=60, supersample=2, kernel=kernel)
phys_model = PhysicalModel([epl.EPL(50), shear.Shear()], [sersic.SersicEllipse(use_lstsq=False)], [sersic.SersicEllipse(use_lstsq=False)])
lens_sim = LensSimulator(phys_model, sim_config, bs=1)

save_prefix = f'FitSaves/FitResults60px'

In [6]:
indices = [0]#list(range(len(observed_imgs)))

skip_indices = []#[4, 27]

for i in indices:
    print(i)
    img = observed_imgs[i]
    if i in skip_indices:
        print("Skipping because of previous failure to converge")
        continue
    
    results = simulate_system(img, prior, ModellingSequence, sim_config, phys_model, lens_sim, map_steps=350)
    # results = {"map_best":map_best, "SVI_samples":SVI_samples, "HMC_samples":HMC_samples, "HMC_median":HMC_median, "HMC_img":HMC_img,
    #            "map_chisq_hist":map_chisq_hist, "svi_loss_hist":svi_loss_hist}
    
    # filename = f'{save_prefix}_{i}.pkl'
    # with open(filename, 'wb') as file:
    #     pickle.dump(results, file)
    

0
Starting MAP


Chi-squared: 1.033: 100%|██████████| 350/350 [00:26<00:00, 13.23it/s] 


Starting SVI


ELBO: -6.012: 100%|██████████| 1500/1500 [01:28<00:00, 16.98it/s] 


Starting HMC
Sampling took 72.0s


In [7]:
results['HMC'].HMC_rhat

Array([1.003408 , 1.0012431, 1.0363746, 1.0078332, 1.0421777, 1.0206746,
       1.0144774, 1.0064628, 1.0028504, 1.0027279, 1.0009372, 1.0005323,
       1.000013 , 1.0012493, 1.0029238, 1.0045134, 1.0215322, 1.0373538,
       1.0382172, 1.0059228, 1.0008801, 1.0035137], dtype=float32)

In [8]:
c = qz.covariance()

NameError: name 'qz' is not defined

In [ ]:
ev = jnp.linalg.eigvalsh(c)
jnp.min(ev), jnp.max(ev)

In [ ]:
# refit_idx = 56
# res = simulate_system(observed_imgs[refit_idx], prior, sim_config, 
#                       phys_model, lens_sim, start_svi_from_truth=True, 
#                       true_params=index_params(true_params, refit_idx), map_steps=350, n_vi=10000)


# with open(f'{save_prefix}_{refit_idx}_SVITruthVI10000.pkl', 'wb') as file:
#     pickle.dump(res, file)

In [ ]:
i = 0
filename = f'{save_prefix}_{i}.pkl'
with open(filename, 'rb') as file:
    r = pickle.load(file)

display_results(r, observed_imgs[i], index_params(true_params, i), lens_sim)

In [ ]:

i = 54
filename = f'FitSaves/FitResults_{i}.pkl'
with open(filename, 'rb') as file:
    r_original = pickle.load(file)

filename = f'FitSaves/ReFitResults_{i}.pkl'
with open(filename, 'rb') as file:
    r_refit = pickle.load(file)

filename = f'FitSaves/ReFitResults_{i}_MAP5600.pkl'
with open(filename, 'rb') as file:
    r_refit2 = pickle.load(file)

plt.title("SVI Curves with different MAP starting points")
plt.plot(r_original['svi_loss_hist'], label=f"350 MAP Steps")
plt.plot(r_refit['svi_loss_hist'], label=f"1400 MAP Steps")
plt.plot(r_refit2['svi_loss_hist'], label=f"5600 MAP Steps")
plt.legend()
plt.yscale('log')
plt.show()


In [ ]:
schedule_fn = optax.polynomial_schedule(init_value=-1e-6, end_value=-3e-3,
                                          power=2, transition_steps=300)

dummy_x = np.arange(1500)
plt.plot(dummy_x, schedule_fn(dummy_x))
plt.title("SVI Schedule Function")
plt.show()

In [ ]:
from scipy.signal import correlate2d
autocorr = correlate2d(observed_imgs[0], norm_residuals[0], mode='full', boundary='symm')
autocorr /= np.max(autocorr)  # normalize
plt.imshow(autocorr)